### Softmax evaluation for large number of neurons.

The softmax function is multinomial distribution function where we evaluate probability of the class with all classes in mind. It's defined as for n neurons

$$f(x) = \frac {e^{x_i}} {\sum\limits_{j=1}^n {e^{x_j}}}$$

Since the $e^x_i$ is positive the sum of the values surely overflow hence is there any alternative out there. Let's check out.

In [1]:
#import the numpy for vectorized evaluations.
import numpy as np

#lets have a moderate size numpy array.
a = np.random.rand(100000)

#compute it's exponent
ae = np.exp(a)

Evaluate the softmax using bruteforce way.

In [2]:
#compute the softmax
bsm = ae / np.sum(ae)

Let's evaluate the softmax using a hack. We will divide all the elements by some constant (we choose the largest of the exponent as a divider) and try to evaluate the softmax. Will it work?

In [3]:
#divide the array using maximum of exponents.
fae = ae / np.max(ae)

#compute softmax now.
tsm = fae / np.sum(fae)

Now let's compare the results. Since we have compromised the precision we cannot compare them programmatically let our brains do it by examining a slice of the probabilities.

In [4]:
print("bruteforce softmax:", bsm[:3])
print("alternate softmax:", tsm[:3])

bruteforce softmax: [  6.63381613e-06   6.88408896e-06   9.27157681e-06]
alternate softmax: [  6.63381613e-06   6.88408896e-06   9.27157681e-06]


Whoa! we did it. The softmax can be now calculated without causing overflow. This was a need when I was implementing the `word2vec` model. But how did it work? Damn simple.. Have a look.

__Normal softmax__
$$ f(x) = \frac {e^{x_i}} {\sum\limits _{j = 0} ^{n} {e^{x_j}} } $$

__Alternate softmax__.  
let's say the constant we choose be $k$
$$
f(x) = \frac{\frac{e^{x_i}}{k}}{\sum\limits_{j=0}^{n}{\frac{e^{x_j}}{k}}} \\
     = \frac{\left(\frac{1}{k}\right)}{\left(\frac{1}{k}\right)} \times \left(\frac {e^{x_i}} {\sum\limits_{j=0}^n{e^{x_j}}}\right) \\
     \text {Both $\left(\frac{1}{k} \right)$ cancel out and we are left with.} \\
     f(x) = \frac {e^{x_i}} {\sum\limits _{j = 0} ^{n} {e^{x_j}} } 
$$

Isn't it the same softmax we want? But there is a loss of precision possible out there while dividing with k. If that's fine we can go ahead with this. But wait what should be the value of $k$ . Here we choose $k$ as maximum of elements in the vector. But is it going to be helpful all the time? Well for now, I don't have any answer.